### Text classification using attention (Seq to One)
We use a bidirectional LSTM as encoder and an attention layer

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchtext
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn import model_selection
from sklearn import metrics
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
import re
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
import pytorch_lightning as pl
from torch.nn.functional import binary_cross_entropy_with_logits, binary_cross_entropy
from torchmetrics import Accuracy, F1
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
import string
import statistics
import nltk

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/bk_anupam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/bk_anupam/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Configuration for training

In [3]:
class MODEL_EVAL_METRIC:
    accuracy = "accuracy"
    f1_score = "f1_score"

class Config:
    VOCAB_SIZE = 0
    BATCH_SIZE = 256
    EMB_SIZE = 300
    OUT_SIZE = 2
    NUM_FOLDS = 5
    NUM_EPOCHS = 20
    NUM_WORKERS = 8
    # Whether to update the pretrained embedding weights during training process
    EMB_WT_UPDATE = True
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    MODEL_EVAL_METRIC = MODEL_EVAL_METRIC.accuracy
    FAST_DEV_RUN = False    
    PATIENCE = 6    
    IS_BIDIRECTIONAL = True
    # model hyperparameters
    MODEL_HPARAMS = {
        "hidden_size": 141, 
        "num_layers": 2,         
        "drop_out": 0.4258,
        "lr": 0.000366,
        "weight_decay": 0.00001
    }

# For results reproducibility 
# sets seeds for numpy, torch, python.random and PYTHONHASHSEED.
pl.seed_everything(42, workers=True)

Global seed set to 42


42

### Load the data

In [4]:
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')
print(f"Rows in train.csv = {len(df_train)}")
print(f"Rows in test.csv = {len(df_test)}")
pd.set_option('display.max_colwidth', None)
df_train.head()

Rows in train.csv = 7613
Rows in test.csv = 3263


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


### Some EDA

In [5]:
df_train_pos = df_train[df_train.target == 1]
df_train_neg = df_train[df_train.target == 0]
print(f"No. of positive training examples = {len(df_train_pos)}")
print(f"No. of negative training examples = {len(df_train_neg)}")
train_keywords_unique = df_train.keyword.unique()
print(f"No. of unique keywords = {len(train_keywords_unique)}")
df_train_notnull_keywords = df_train[~df_train.keyword.isnull()]
print(f"No of train examples with keyword not null = {len(df_train_notnull_keywords)}")

No. of positive training examples = 3271
No. of negative training examples = 4342
No. of unique keywords = 222
No of train examples with keyword not null = 7552


### K Fold CV

In [6]:
# split the training dataframe into kfolds for cross validation. We do this before any processing is done
# on the data. We use stratified kfold if the target distribution is unbalanced
def strat_kfold_dataframe(df, target_col_name, num_folds=5):
    # we create a new column called kfold and fill it with -1
    df["kfold"] = -1
    # randomize of shuffle the rows of dataframe before splitting is done
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    # get the target data
    y = df["target"].values
    skf = model_selection.StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
    for fold, (train_index, val_index) in enumerate(skf.split(X=df, y=y)):
        df.loc[val_index, "kfold"] = fold
    return df        

df_train = strat_kfold_dataframe(df_train, target_col_name="target", num_folds=5)    

### Tweet preprocessing

In [7]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~`" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
def clean_special_chars(text, punct):
    for p in punct:
        text = text.replace(p, ' ')
    return text

def process_tweet(df, text, keyword):
    lemmatizer = WordNetLemmatizer()    
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)    
    processed_text = []
    stop = stopwords.words("english")
    for tweet, keyword in zip(df[text], df[keyword]):
        tweets_clean = []        
        # remove stock market tickers like $GE
        #tweet = tweet + " " + keyword
        tweet = re.sub(r'\$\w*', '', tweet)
        # remove old style retweet text "RT"
        tweet = re.sub(r'^RT[\s]+', '', tweet)
        # remove hyperlinks
        tweet = re.sub(r'http\S+', '', tweet)
        # remove hashtags
        # only removing the hash #, @, ... sign from the word
        tweet = re.sub(r'\.{3}|@|#', '', tweet)    
        tweet = clean_special_chars(tweet, punct)
        # remove junk characters which don't have an ascii code
        tweet = tweet.encode("ascii", "ignore").decode("utf-8", "ignore")
        # tokenize tweets        
        tweet_tokens = tokenizer.tokenize(tweet)
        for word in tweet_tokens:
            # remove stopwords and punctuation
            #if (word.isalpha() and len(word) > 2 and word not in stop 
            #    and word not in string.punctuation):
                #stem_word = stemmer.stem(word)  # stemming word            
                #lem_word = lemmatizer.lemmatize(word)
                #tweets_clean.append(lem_word) 
                tweets_clean.append(word)
        processed_text.append(" ".join(tweets_clean))        
    df['processed_text'] = np.array(processed_text)

In [8]:
# Fill in missing values
df_train["keyword"] = df_train["keyword"].fillna("no_keyword")
df_test["keyword"] = df_test["keyword"].fillna("no_keyword")
process_tweet(df_train, 'text', "keyword")
process_tweet(df_test, 'text', "keyword")
# length of the processed tweet
df_train["prcsd_tweet_len"] = df_train["processed_text"].apply(lambda row: len(row.split()))
df_test["prcsd_tweet_len"] = df_test["processed_text"].apply(lambda row: len(row.split()))
df_train.iloc[50:52, :]

,id,keyword,location,text,target,kfold,processed_text,prcsd_tweet_len
50,5174,fatalities,Official Website,#HSE releases annual workplace facilities data. Have a look | http://t.co/h4UshEekxm http://t.co/jNHNX3oISN,0,4,hse releases annual workplace facilities data have a look,9
51,3126,debris,NaN,#??? #?? #??? #??? MH370: Aircraft debris found on La Reunion is from missing Malaysia Airlines ... http://t.co/zxCORQ0A3a,1,0,mh370 aircraft debris found on la reunion is from missing malaysia airlines,12


### Model building starts from here

In [9]:
# Load the GloVe word embedding for tweets
#emb = torchtext.vocab.GloVe(name="twitter.27B", dim=200)
emb = torchtext.vocab.FastText(language="en")

In [10]:
# build tweets vocab from training data
def yield_tokens(df):
    for index, row in df.iterrows():
        yield row["processed_text"].split()
    
tweet_vocab = build_vocab_from_iterator(yield_tokens(df_train), specials=["<unk>", "<pad>"])   
Config.VOCAB_SIZE = len(tweet_vocab)

In [11]:
# For the problem specific vocab, get the embedding vectors from the pre-trained embedding
# for each word in vocab and return a matrix of shape vocab_size, embedding_dim. This matrix
# will be the pretrained embedding weight matrix which we will use to create the embedding layer
def get_vocab_pt_emb_matrix(text_vocab, emb):
    embedding_matrix = []
    for token in text_vocab.get_itos():
        embedding_matrix.append(emb[token])
    return torch.stack(embedding_matrix)

pt_emb_weights = get_vocab_pt_emb_matrix(tweet_vocab, emb)
pt_emb_layer = nn.Embedding.from_pretrained(pt_emb_weights)

In [12]:
# vectorize the processed tweet, i.e. replace each token in the tweet with its corresponding index
# in the tweet vocab
df_train["vectorized_tweet"] = df_train["processed_text"].apply(
    lambda row:torch.LongTensor(tweet_vocab.lookup_indices(row.split()))
    )

### Tweet dataset

In [13]:
class VectorizedTweetDataSet(Dataset):
    def __init__(self, tweet_vecs, labels):
        self.tweet_vecs = tweet_vecs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        tweet_vec = self.tweet_vecs[idx]
        label = self.labels[idx]
        tweet_len = len(tweet_vec)
        return (tweet_vec, label)


### Get train and validation data for a fold

In [14]:
def get_fold_dls(fold, df):
    train_df = df[df.kfold != fold].reset_index(drop=True)
    valid_df = df[df.kfold == fold].reset_index(drop=True)
    X_train = train_df["vectorized_tweet"].to_numpy()
    y_train = train_df["target"].to_numpy()
    X_valid = valid_df["vectorized_tweet"].to_numpy()
    y_valid = valid_df["target"].to_numpy()
    ds_train = VectorizedTweetDataSet(X_train, y_train)
    ds_valid = VectorizedTweetDataSet(X_valid, y_valid)
    dl_train = DataLoader(ds_train, batch_size=Config.BATCH_SIZE, shuffle=True, collate_fn=pad_collate, num_workers=Config.NUM_WORKERS)
    dl_valid = DataLoader(ds_valid, batch_size=Config.BATCH_SIZE, collate_fn=pad_collate, num_workers=Config.NUM_WORKERS)
    return dl_train, dl_valid

### Pad the input sequence

In [15]:
# If the goal is to train with mini-batches, one needs to pad the sequences in each batch. 
# In other words, given a mini-batch of size N, if the length of the largest sequence is L, 
# one needs to pad every sequence with a length of smaller than L with zeros and make their 
# lengths equal to L. Moreover, it is important that the sequences in the batch are in the 
# descending order.
def pad_collate(batch):
    # Each element in the batch is a tuple (data, label)
    # sort the batch (based on tweet word count) in descending order
    sorted_batch = sorted(batch, key=lambda x:x[0].shape[0], reverse=True)
    sequences = [x[0] for x in sorted_batch]
    sequences_padded = pad_sequence(sequences, batch_first=True, padding_value=0)
    # Also need to store the length of each sequence.This is later needed in order to unpad 
    # the sequences
    seq_len = torch.Tensor([len(x) for x in sequences])
    labels = torch.Tensor([x[1] for x in sorted_batch])
    return sequences_padded, seq_len, labels


## Model architecture

<img src="temp.jpg" style="width:800px;height:500px;">

### Build the model 
Encoder (biLSTM) => Attention layer => Fully connected layer => Sigmoid 

**Bidirectional RNN as encoder** <br>
outputs is of size [src len, batch size, hid dim * num directions] where the first hid_dim elements in the third axis are the hidden states from the top layer forward RNN, and the last hid_dim elements are hidden states from the top layer backward RNN. We can think of the third axis as being the forward and backward hidden states concatenated together other. 

hidden is of size [n layers * num directions, batch size, hid dim], where [-2, :, :] gives the top layer forward RNN hidden state after the final time-step (i.e. after it has seen the last word in the sentence) and [-1, :, :] gives the top layer backward RNN hidden state after the final time-step (i.e. after it has seen the first word in the sentence).

The bidirectional rnn encoder returns the hidden state from each time step as well as the final hidden state (last time step). 

**Attention layer** <br>
Takes as input encoder outputs ( hidden state from each time step of last rnn layer) as well encoder final hidden state (fom the last time step).
encoder_outputs + enc_final_hidden_state => alignment_score (use one of the methods below. We use concat and dot product implementation of alignment score)

<img src="align_score.jpg" style="width:800px;height:500px;">

Softmax(alignment_score) => attention weights <br>
attention weights * encoder outputs => context vector <br>
Context vector has dimensions batch_size, hidden_size and is return by the attention layer <br>

**FC layer** <br>
Input => Context vector (batch_size, hidden_size) <br>
Output => batch_size, out_size

In [16]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class Encoder(nn.Module):    
    def __init__(self, params, hparams):
        super().__init__()                
        self.num_layers = hparams["num_layers"]
        self.hidden_size = hparams["hidden_size"]
        self.is_bidirect = params["is_bidirect"]
        self.num_directions = 2 if self.is_bidirect else 1
        # Embedding layer
        self.emb_layer = nn.Embedding(params["vocab_size"], params["emb_size"])
        # copy the vocab specific weights(emb vectors) from pretrained embeddings to model embedding layer
        self.emb_layer.weight.data.copy_(params["pt_emb_weights"])
        # whether to update the pretrained embedding layer weights during model training
        self.emb_layer.weight.requires_grad = params["emb_wt_update"] 
        # LSTM Layer        
        self.lstm_layer = nn.LSTM(
                        input_size=params["emb_size"], 
                        hidden_size=self.hidden_size, 
                        batch_first=True, 
                        bidirectional=self.is_bidirect, 
                        num_layers=self.num_layers, 
                        dropout=hparams["drop_out"]
                        )
        
    def forward(self, inputs, input_lengths, state):        
        # inputs = [batch_size, batch_max_seq_length]        
        # embeds is of shape batch_size * num_steps * emb_dim and is the input to lstm layer
        embeds = self.emb_layer(inputs)        
        # final hidden state (from last time step)
        h_final = None        
        # embeds = [batch_size, max_seq_length, emb_dim]        
        embeds_pack = pack_padded_sequence(embeds, input_lengths.to("cpu"), batch_first=True)                
        lstm_out_pack, (h_n, c_n) = self.lstm_layer(embeds_pack)
        # h_n and c_n = [num_directions * num_layers, batch_size, hidden_size]
        # unpack the output
        lstm_out, lstm_out_len = pad_packed_sequence(lstm_out_pack, batch_first=True)        
        # print(f"lstm_out.shape = {lstm_out.shape}") # [batch_size, max_seq_length, hidden_size * num_directions]        
        if self.is_bidirect:                        
            h_tend_fwd = h_n[-2, :, :]
            h_tend_bwd = h_n[-1, :, :]
            h_final = torch.cat((h_tend_fwd, h_tend_bwd), dim=1)            
        else:                        
            h_final = h_n[-1, :, :]   

        # print(f"h_final.shape = {h_final.shape}") # [batch_size, hidden_size * num_directions]
        return lstm_out, lstm_out_len, h_final

    def init_state(self, batch_size=1):
        """ Initialize the hidden state i.e. initialize all the neurons in all the hidden layers 
        to zero"""
        if not isinstance(self.lstm_layer, nn.LSTM):
            # `nn.GRU` takes a tensor as hidden state
            return torch.zeros((self.num_directions * self.num_layers, batch_size, self.hidden_size))
        else:
            # `nn.LSTM` takes a tuple of hidden states (h0, c0). h0 = initial
            # hidden state for each element in the batch, c0 = initial cell state
            # for each element in the batch
            return (torch.zeros((self.num_directions * self.num_layers, batch_size, self.hidden_size)),
                    torch.zeros((self.num_directions * self.num_layers, batch_size, self.hidden_size)))

In [17]:
class AttentionMethod:
    CONCAT = "concat"
    DOT = "dot"
    GENERAL = "general"

In [18]:
class AttentionLayer(nn.Module):
    def __init__(self, hidden_size, attn_method = AttentionMethod.CONCAT):
        super().__init__()
        self.hidden_size = hidden_size
        self.attn_method = attn_method   
        self.attn = None
        self.v = None
        self.concat_linear = None
        if self.attn_method == AttentionMethod.CONCAT:     
            self.attn = nn.Linear((hidden_size * 2) + (hidden_size * 2), hidden_size)
            self.v = nn.Linear(hidden_size, 1, bias=False)
        if self.attn_method == AttentionMethod.DOT:
            self.attn = nn.Linear((hidden_size * 2) + (hidden_size * 2), hidden_size)

    def attn_concat(self, encoder_outputs, enc_final_hidden_state):        
        batch_size, seq_length, _ = encoder_outputs.shape        
        # add seq_length dim to enc_final_hiddden_state
        enc_final_hidden_state = enc_final_hidden_state.unsqueeze(1)
        # enc_final_hidden_state.shape = [batch_size, 1, enc_hidden_size * 2]
        # now repeat the final hidden state seq_length times across dim 1 so final hidden state and encoder outputs have same dimensions
        enc_final_hidden_state = enc_final_hidden_state.repeat(1, seq_length, 1)
        # print(f"enc_final_hidden_state.shape = {enc_final_hidden_state.shape}") # [batch_size, seq_length, enc_hidden_size * 2]
        # concat the enc final hidden state and the encoder outputs (which are nothing but enc hidden states at individual time steps)
        energy = torch.tanh(self.attn(torch.cat((encoder_outputs, enc_final_hidden_state), dim=2)))
        # print(f"energy.shape = {energy.shape}") # [batch_size, seq_length, enc_hidden_size]
        # get attention vector corresponding to each source time step
        attention = self.v(energy).squeeze(2)
        return attention

    def attn_dot(self, encoder_outputs, enc_final_hidden_state):        
        attention = torch.bmm(encoder_outputs, enc_final_hidden_state.unsqueeze(2))
        # print(f"attention.shape = {attention.shape}") # [batch_size, seq_length, 1]
        return attention.squeeze(2)

    def forward(self, encoder_outputs, enc_final_hidden_state):        
        # print(f"encoder_outputs.shape = {encoder_outputs.shape}") #[batch_size, seq_length, enc_hidden_size * num_directions]
        # print(f"enc_final_hidden_state.shape = {enc_final_hidden_state.shape}") # [batch_size, enc_hidden_size * num_directions]        
        attention = None
        if self.attn_method == AttentionMethod.CONCAT:
            attention = self.attn_concat(encoder_outputs, enc_final_hidden_state)
        elif self.attn_method == AttentionMethod.DOT:
            attention = self.attn_dot(encoder_outputs, enc_final_hidden_state)            
        # print(f"(attention.shape = {attention.shape}") # [batch_size, seq_length]
        attn_weights = F.softmax(attention, dim=1)
        # print(f"(attn_weights.shape = {attn_weights.shape}") # [batch_size, seq_length]
        attn_weights = attn_weights.unsqueeze(1)
        # print(f"(attn_weights.shape = {attn_weights.shape}") # [batch_size, 1, seq_length]
        # apply attention weights to encoder outputs to get context vector
        context_vector = torch.bmm(attn_weights, encoder_outputs)
        # print(f"(context_vector.shape = {context_vector.shape}") # [batch_size, 1, enc_hidden_size * 2]
        # attn_hidden = torch.tanh(self.attn(torch.cat((context_vector.squeeze(1), enc_final_hidden_state), dim=1)))
        # print(f"attn_hidden.shape = {attn_hidden.shape}") # [batch_size, hidden_size]
        return context_vector.squeeze(1)

In [19]:
class RnnAttnClassifier(nn.Module):
    def __init__(self, params, hparams):
        super().__init__()
        self.encoder = Encoder(params, hparams)
        self.attention_layer = AttentionLayer(hparams["hidden_size"], params["attn_method"])
        self.fc = None
        if params["attn_method"] == AttentionMethod.CONCAT:
            self.fc = nn.Linear(hparams["hidden_size"] * 2, 2)
        elif params["attn_method"] == AttentionMethod.DOT:
            self.fc = nn.Linear(hparams["hidden_size"] * 2, 2)
        self.act = nn.Sigmoid()

    def forward(self, inputs, input_lengths, state):
        enc_out, enc_out_len, enc_h_final = self.encoder(inputs, input_lengths, state)
        ctx_vec = self.attention_layer(enc_out, enc_h_final)
        out = self.fc(ctx_vec)
        return self.act(out)

    def init_state(self):
        return self.encoder.init_state()

### Pytorch lightning wrapper for model

In [20]:
class DisasterTweetLitModel(pl.LightningModule):
    def __init__(self, params, hparams, model_eval_metric=MODEL_EVAL_METRIC.accuracy):
        super().__init__()
        #self.save_hyperparameters()
        self.lr = hparams["lr"]
        self.weight_decay = hparams["weight_decay"]
        self.model_eval_metric = model_eval_metric
        self.network = RnnAttnClassifier(params, hparams)            

    def forward(self, tweets, tweet_lengths, state):
        return self.network(tweets, tweet_lengths, state)

    def configure_optimizers(self):
        model_optimizer = torch.optim.Adam(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(model_optimizer, mode="min")
        return {
            "optimizer": model_optimizer,
            "lr_scheduler": {
                "scheduler": lr_scheduler,
                "monitor": "val_loss",
                "frequency": 1
            }
        }

    def training_step(self, batch, batch_idx):
        tweets, tweet_lengths, targets = batch
        # initialize the hidden and cell state of the LSTM
        h0, c0 = self.network.init_state()
        targets_pred = self(tweets, tweet_lengths, (h0, c0))        
        #print(f"targets_pred.shape = {targets_pred.shape}")
        loss_targets = F.one_hot(targets.T.long(), num_classes=2)
        loss_targets = loss_targets.float()        
        train_loss = binary_cross_entropy(targets_pred, loss_targets)
        train_metric = None
        train_metric_str = ""
        if self.model_eval_metric == MODEL_EVAL_METRIC.accuracy:            
            targets_pred = torch.argmax(targets_pred, dim=1)            
            train_metric = Accuracy(num_classes=2)(targets_pred.cpu(), targets.long().cpu())
            train_metric_str = "train_acc"
        elif self.model_eval_metric == MODEL_EVAL_METRIC.f1_score:
            train_metric = F1(targets_pred, targets)            
            train_metric_str = "train_f1"
        self.log("train_loss", train_loss, prog_bar=True, logger=True, on_epoch=True, on_step=True)
        self.log(train_metric_str, train_metric, prog_bar=True, logger=True, on_epoch=True, on_step=True)
        return train_loss

    def validation_step(self, batch, batch_idx):
        tweets, tweet_lengths, targets = batch
        # initialize the hidden and cell state of the LSTM
        h0, c0 = self.network.init_state()
        targets_pred = self(tweets, tweet_lengths, (h0, c0))
        loss_targets = F.one_hot(targets.T.long(), num_classes=2)
        loss_targets = loss_targets.float()        
        val_loss = binary_cross_entropy(targets_pred, loss_targets)
        val_metric = None
        val_metric_str = ""
        if self.model_eval_metric == MODEL_EVAL_METRIC.accuracy:
            targets_pred = torch.argmax(targets_pred, dim=1)
            val_metric = Accuracy(num_classes=2)(targets_pred.cpu(), targets.long().cpu())
            val_metric_str = "val_acc"
        elif self.model_eval_metric == MODEL_EVAL_METRIC.f1_score:
            val_metric = F1(targets_pred, targets)            
            val_metric_str = "val_f1"
        self.log("val_loss", val_loss, prog_bar=True, logger=True, on_epoch=True, on_step=True)
        self.log(val_metric_str, val_metric, prog_bar=True, logger=True, on_epoch=True, on_step=True)
        return val_loss

### Custom lightning callback 
To record training and validation metric values at each epoch and the best metric values across all epochs

In [21]:
from pytorch_lightning.callbacks import Callback
from pytorch_lightning import LightningModule, Trainer
# Monitor multiple metric values that are calculated either in training or validation step and return the
# best metric values for each epoch
class MetricsAggCallback(Callback):
    def __init__(self, train_metrics_to_monitor, val_metrics_to_monitor):
        # dictionary with metric name as key and monitor mode (min, max) as the value
        # ( the same names used to log metric values in training and validation step)
        self.val_metrics_to_monitor = val_metrics_to_monitor
        self.train_metrics_to_monitor = train_metrics_to_monitor
        # dictionary with metric_name as key and list of metric value for each epoch
        self.train_metrics = {metric: [] for metric in train_metrics_to_monitor.keys()}
        self.val_metrics = {metric: [] for metric in val_metrics_to_monitor.keys()}
        # dictionary with metric_name as key and the best metric value for all epochs
        self.train_best_metric = {metric: None for metric in train_metrics_to_monitor.keys()}
        self.val_best_metric = {metric: None for metric in val_metrics_to_monitor.keys()}
        # dictionary with metric_name as key and the epoch number with the best metric value
        self.train_best_metric_epoch = {metric: None for metric in train_metrics_to_monitor.keys()}     
        self.val_best_metric_epoch = {metric: None for metric in val_metrics_to_monitor.keys()}     
        self.epoch_counter = 0           

    @staticmethod
    def process_metrics(metrics_to_monitor, metrics, best_metric, best_metric_epoch, trainer):
        metric_str = ""
        for metric, mode in metrics_to_monitor.items():
            metric_value = round(trainer.callback_metrics[metric].cpu().detach().item(), 4)            
            metric_str += f"{metric} = {metric_value}, "
            metrics[metric].append(metric_value)
            if mode == "max":
                best_metric[metric] = max(metrics[metric])            
            elif mode == "min":            
                best_metric[metric] = min(metrics[metric])            
            best_metric_epoch[metric] = metrics[metric].index(best_metric[metric]) 
        print(metric_str[:-2])

    def on_train_epoch_end(self, trainer: Trainer, pl_module: LightningModule):
        self.epoch_counter += 1        
        self.process_metrics(self.train_metrics_to_monitor, self.train_metrics, self.train_best_metric, self.train_best_metric_epoch, trainer)

    def on_validation_epoch_end(self, trainer: Trainer, pl_module: LightningModule):        
        print(f"For epoch {self.epoch_counter}")
        self.process_metrics(self.val_metrics_to_monitor, self.val_metrics, self.val_best_metric, self.val_best_metric_epoch, trainer)


In [22]:
model_params = {
        "vocab_size": Config.VOCAB_SIZE,
        "emb_size": Config.EMB_SIZE,
        "output_size": Config.OUT_SIZE,
        "pt_emb_weights": pt_emb_weights,
        "emb_wt_update": Config.EMB_WT_UPDATE,
        "is_bidirect": True,
        "attn_method": AttentionMethod.DOT
    }

In [23]:
def run_training(fold, dl_train, dl_val, find_lr=True):
    fold_str = f"fold{fold}"
    print(f"Running training for {fold_str}")    
    disaster_tweet_model = DisasterTweetLitModel(
        params=model_params,        
        hparams=Config.MODEL_HPARAMS,
        model_eval_metric=Config.MODEL_EVAL_METRIC                
        )
    tb_logger = pl.loggers.TensorBoardLogger(save_dir="logs")    
    chkpt_file_name = fold_str + "_best_model_{epoch}_{val_loss:.4f}"
    train_metrics_to_monitor = {
        "train_loss": "min",
        "train_acc": "max"
    }
    val_metrics_to_monitor = {
        "val_loss": "min",
        "val_acc": "max",
        }
    loss_chkpt_callback = ModelCheckpoint(dirpath="./model", verbose=True, monitor="val_loss", mode="min", filename=chkpt_file_name)    
    metric_chkpt_callback = MetricsAggCallback(train_metrics_to_monitor, val_metrics_to_monitor)
    early_stopping_callback = EarlyStopping(monitor="val_loss", patience=Config.PATIENCE, mode="min", verbose=True)
    trainer = pl.Trainer(
        gpus = 1,
        deterministic = True,
        auto_select_gpus = True,
        progress_bar_refresh_rate = 20,
        max_epochs = Config.NUM_EPOCHS,
        logger = tb_logger,
        auto_lr_find = True,    
        #precision = Config.PRECISION,   
        fast_dev_run = Config.FAST_DEV_RUN, 
        gradient_clip_val = 1.0,        
        callbacks = [loss_chkpt_callback, metric_chkpt_callback, early_stopping_callback]
    )        
    if find_lr:
        trainer.tune(model=disaster_tweet_model, train_dataloaders=dl_train)
        print(disaster_tweet_model.lr)
    trainer.fit(disaster_tweet_model, train_dataloaders=dl_train, val_dataloaders=dl_val)
    fold_train_metrics = {
        metric: (metric_chkpt_callback.train_best_metric[metric], metric_chkpt_callback.train_best_metric_epoch[metric]) 
        for metric in train_metrics_to_monitor.keys()
    }
    fold_val_metrics = {
        metric: (metric_chkpt_callback.val_best_metric[metric], metric_chkpt_callback.val_best_metric_epoch[metric]) 
        for metric in val_metrics_to_monitor.keys()
    }            
    del trainer, disaster_tweet_model, loss_chkpt_callback, metric_chkpt_callback 
    return fold_train_metrics, fold_val_metrics

In [24]:
find_lr = True
all_fold_val_loss = []
all_fold_val_acc = []

for fold in range(Config.NUM_FOLDS):
    dl_train, dl_val = get_fold_dls(fold, df_train)
    fold_train_metrics, fold_val_metrics = run_training(fold, dl_train, dl_val, find_lr=False)    
    all_fold_val_loss.append(fold_val_metrics["val_loss"][0])
    all_fold_val_acc.append(fold_val_metrics["val_acc"][0])
    print(f"Best train metrics values for fold{fold}")    
    print(fold_train_metrics)
    print(f"Best val metrics values for fold{fold}")    
    print(fold_val_metrics)                

Running training for fold0


/home/bk_anupam/anaconda3/envs/fastai/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:446: UserWarning: Checkpoint directory ./model exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type              | Params
----------------------------------------------
0 | network | RnnAttnClassifier | 6.2 M 
----------------------------------------------
6.2 M     Trainable params
0         Non-trainable params
6.2 M     Total params
24.781    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


For epoch 0
val_loss = 0.6932, val_acc = 0.5215


...

Epoch 7, global step 191: val_loss was not in top 1


For epoch 7
val_loss = 0.5408, val_acc = 0.7996
train_loss = 0.203, train_acc = 0.931


Validating: 0it [00:00, ?it/s]

Monitored metric val_loss did not improve in the last 6 records. Best score: 0.446. Signaling Trainer to stop.
Epoch 8, global step 215: val_loss was not in top 1


For epoch 8
val_loss = 0.6435, val_acc = 0.7884
train_loss = 0.1659, train_acc = 0.9453
Best train metrics values for fold4
{'train_loss': (0.1659, 8), 'train_acc': (0.9453, 8)}
Best val metrics values for fold4
{'val_loss': (0.4458, 3), 'val_acc': (0.8127, 5)}


In [25]:
#all_fold_val_loss = [x[0] for x in all_fold_val_loss]
#all_fold_val_acc = [x[0] for x in all_fold_val_acc]
print(f"val loss across folds = {all_fold_val_loss}")
print(f"val accuracy across folds = {all_fold_val_acc}")
mean_loss = statistics.mean(all_fold_val_loss)
mean_acc = statistics.mean(all_fold_val_acc)
std_loss = statistics.stdev(all_fold_val_loss)
std_acc = statistics.stdev(all_fold_val_acc)
print(f"mean val loss across folds = {mean_loss}, val loss stdev across fold = {std_loss}")
print(f"mean val accuracy across folds = {mean_acc}, val accuracy stdev across fold = {std_acc}")

val loss across folds = [0.409, 0.4359, 0.4328, 0.4206, 0.4458]
val accuracy across folds = [0.8391, 0.828, 0.8306, 0.8285, 0.8127]
mean val loss across folds = 0.42882, val loss stdev across fold = 0.014271369941249515
mean val accuracy across folds = 0.82778, val accuracy stdev across fold = 0.00954028301466995


In [26]:
best_model = DisasterTweetLitModel.load_from_checkpoint(
    checkpoint_path="./model/fold0_best_model_epoch=4_val_loss=0.4090.ckpt",
    params = model_params,
    hparams = Config.MODEL_HPARAMS
    )
print(best_model)
tweet_vocab.set_default_index(0)
df_test["vectorized_tweet"] = df_test["processed_text"].apply(
    lambda row:torch.LongTensor(tweet_vocab.lookup_indices(row.split()))
    )

# Do prediction with best performing model on the test set
def predict(df_test):
    test_output = []
    for index, row in df_test.iterrows():    
        vec_tweet = row["vectorized_tweet"]
        if len(vec_tweet) == 0:
            test_output.append(0)
            continue
        vec_tweet_len = torch.IntTensor([len(vec_tweet)])
        vec_tweet = vec_tweet.view(1, -1)    
        #print(vec_tweet, vec_tweet_len)
        output = best_model(vec_tweet, vec_tweet_len, state=None)
        #print(output)
        test_output.append(torch.argmax(output).item())    
    return test_output        

test_output = predict(df_test)
print(len(test_output))

df_submission = pd.read_csv('./data/submission.csv')
df_submission['target']= test_output
df_submission.to_csv('my_submission.csv',index=False)

DisasterTweetLitModel(
  (network): RnnAttnClassifier(
    (encoder): Encoder(
      (emb_layer): Embedding(17120, 300)
      (lstm_layer): LSTM(300, 141, num_layers=2, batch_first=True, dropout=0.4258, bidirectional=True)
    )
    (attention_layer): AttentionLayer(
      (attn): Linear(in_features=564, out_features=141, bias=True)
    )
    (fc): Linear(in_features=282, out_features=2, bias=True)
    (act): Sigmoid()
  )
)
3263
